# Run C# and Pwsh in the same Notebook

This notebook will run C# and pwsh in the same notebook and pass values between them

We will see how long it takes between C# and pwsh to count files in a directory. First create 10,000 files

In [95]:
$playdir = "$Env:Temp\pwshplaydir"
if (-not (Test-Path $playdir)) {
 New-Item $playdir -ItemType Directory
    $10k = 0..10000
    $10k | ForEach-Object -ThrottleLimit 10 -Parallel {
        $PlayFileName = $Using:playdir + "\$PSItem.txt"
        $Null = New-Item $PlayFileName -ItemType File -Force
    }
}

Start a stopwatch

In [96]:
$Stopwatch = [diagnostics.stopwatch]::StartNew()

Get the file count with c# using the magic command `#!csharp`

In [97]:
#!csharp
using System.IO; 
using System;
int fileCount = Directory.GetFiles(@"C:\Users\mrrob\AppData\Local\Temp\pwshplaydir", "*", SearchOption.TopDirectoryOnly).Length;
Console.WriteLine($"Number of Files : {fileCount}\n");

Number of Files : 10001



In [98]:
$Stopwatch.Stop()
$csharpmillisecs = $Stopwatch.Elapsed.Milliseconds

Count the same files with pwsh

In [99]:
$Stopwatch = [diagnostics.stopwatch]::StartNew()
$FileCount = (Get-Childitem C:\Users\mrrob\AppData\Local\Temp\pwshplaydir).Count
Write-Output "The File Count is $FileCount"

The File Count is 10001


In [100]:
$Stopwatch.Stop()
$pwshmillisecs = $Stopwatch.Elapsed.Milliseconds

Which was quicker ?

In [101]:
function Which-WasQuicker {
    if($pwshmillisecs -lt $csharpmillisecs ){
        $percquicker = [math]::Round(($pwshmillisecs / $csharpmillisecs)  * 100, 2)
        Write-PSFHostColor -String "<c='blue'> Pwsh did it in</c><c='yellow'> $pwshmillisecs milliseconds</c> which was quicker than C Sharp did it. <c='Magenta'> C Sharp did it in</c><c='yellow'> $csharpmillisecs milliseconds</c> Pwsh completed in $percquicker % of the time " -DefaultColor Green
    }else{
        $percquicker = [math]::Round(($csharpmillisecs / $pwshmillisecs )  * 100, 2)
        Write-PSFHostColor -String "<c='Magenta'> C Sharp did it in</c><c='yellow'> $csharpmillisecs milliseconds</c> which was quicker than Pwsh did.<c='blue'> Pwsh did it in</c><c='yellow'> $pwshmillisecs milliseconds</c> C Sharp completed in $percquicker % of the time" -DefaultColor Green
    }

}

In [102]:
Which-WasQuicker

 Pwsh did it in

 173 milliseconds

 which was quicker than C Sharp did it. 

 C Sharp did it in

 233 milliseconds

 Pwsh completed in 74.25 % of the time 


We can set a variable from C Sharp to the number of files

In [103]:
#!csharp
using System.IO; 
using System;
string value;
string File_Countfromenv;
int fileCount = Directory.GetFiles(@"C:\Users\mrrob\AppData\Local\Temp\pwshplaydir", "*", SearchOption.TopDirectoryOnly).Length;
value = Convert.ToString(fileCount);
Environment.SetEnvironmentVariable("File_Count", value);
File_Countfromenv = Environment.GetEnvironmentVariable("File_Count");
Console.WriteLine($"Number of Files from csharp :{File_Countfromenv}\n");

Number of Files from csharp :10001



and then read it from pwsh

In [104]:
Write-Output "The File Count is $Env:File_Count using variable from csharp in pwsh"

The File Count is 10001 using variable from csharp in pwsh


We can set variables in pwsh and read them in C Sharp

In [105]:
$Env:Play_File_Path = (Get-Item "$env:temp\pwshplaydir").FullName

In [106]:
#!csharp
using System.IO; 
using System;
string value;
string File_Countfromenv;
string File_Path_From_Env;
int fileCount = Directory.GetFiles(Environment.GetEnvironmentVariable("Play_File_Path"), "*", SearchOption.TopDirectoryOnly).Length;
value = Convert.ToString(fileCount);
Environment.SetEnvironmentVariable("File_Count", value);
File_Countfromenv = Environment.GetEnvironmentVariable("File_Count");
File_Path_From_Env = Environment.GetEnvironmentVariable("Play_File_Path");
Console.WriteLine($"Number of Files in the directory {File_Path_From_Env} using csharp :{File_Countfromenv}\n");

Number of Files in the directory C:\Users\mrrob\AppData\Local\Temp\pwshplaydir using csharp :10001



In [107]:
pwd

Path


----


C:\Users\mrrob
